# ReAct (Reason + Act)

El paradigma **ReAct (Reason + Act)** propuesto por **Google**, permite combinar las **capacidad de razonamiento** (ya presentes en técnicas como **CoT**), con la **capacidad de actuar** sobre un medio externo. Esta última parte es la que supone un enorme avance respecto de técnicas limitadas a la descomposición de tareas, como **CoT**. Y es que **ReAct** tiene la habilidad de acceder al mundo exterior para verificar hipótesis, razonar sobre contenido externo o actuar sobre el exterior.



**Paper original**: https://arxiv.org/pdf/2210.03629.pdf

![react](images/react.png)

# ¿Cómo definir las acciones sobre el exterior?

En este caso nos limitaremos a usar la **Wikipedia** como entorno con el que el framework **ReAct** pueda interaccionar. Además, definiremos dos acciones posible:

1. **Search**: Busca una página en concreto en la Wikipedia.
2. **Lookup**: Busca un término en una página en concreto.

### The Hard Way 😥



In [1]:
import wikipedia 
...
# Implementar desde cero las interacciones del LLM con la librería de wikipedia, definir los pasos de Thought, Act, Obs, etc.
# Existe una alternativa?

### The Easy Way 🦜🔗 

In [2]:
from langchain import OpenAI, Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer

Instanciamos las herramientas que podrá usar nuestro agente.

In [3]:
docstore=DocstoreExplorer(Wikipedia())

tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search"
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup"
    )
]

Creamos el LLM, el **cerebro** de nuestro agente.

In [21]:
llm = OpenAI(temperature=0, model_name="text-davinci-003")

Por fortuna, **LangChain** ya tiene agentes **ReAct** para consultas a **DocStores**, de modo que usaremos este. 

In [22]:
react = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True, handle_parsing_errors=True)

In [23]:
question = """
How old will the second daughter of the King of Spain be in 2026?
"""
react.run(question)



> Entering new AgentExecutor chain...

Thought: I need to search the King of Spain and find his second daughter, then find her age and calculate how old she will be in 2026.
Action: Search[King of Spain]
Observation: The monarchy of Spain or Spanish monarchy (Spanish: Monarquía Española) is the constitutional form of government of Spain. It consist of a hereditary monarch that reigns as the head of state, being the highest office of the country. The current monarch is Felipe VI since 19 June 2014, after the abdication of his father, King Juan Carlos I.
The Spanish monarchy is constitutionally referred to as The Crown (Spanish: La Corona), and it comprises the reigning monarch, his or her royal family, and the Royal Household, which supports and facilitates the sovereign in the exercise of his duties and prerogatives. The Royal Family is currently represented by King Felipe VI, Queen Letizia, their daughters Leonor, Princess of Asturias, and Infanta Sofía, and the king's parents, King

'19'

In [26]:
question = """
Which Peter Jackson film features an actor who plays Eddard Stark in Game of Thrones?
"""
react.run(question)



> Entering new AgentExecutor chain...

Thought: I need to search Peter Jackson and Eddard Stark, find the film they are both in, then find the actor who plays Eddard Stark in the film.
Action: Search[Peter Jackson]
Observation: Sir Peter Robert Jackson  (born 31 October 1961) is a New Zealand film director, screenwriter and producer. He is best known as the director, writer and producer of the Lord of the Rings trilogy (2001–2003) and the Hobbit trilogy (2012–2014), both of which are adapted from the novels of the same name by J. R. R. Tolkien. Other notable films include the critically lauded drama Heavenly Creatures (1994), the horror comedy The Frighteners (1996), the epic monster remake film King Kong (2005), the World War I documentary film They Shall Not Grow Old (2018) and the documentary The Beatles: Get Back (2021). He is the fourth-highest-grossing film director of all-time, his films having made over $6.5 billion worldwide.Jackson began his career with the "splatstick" hor

'The Lord of the Rings trilogy'